In [1]:
import sys
sys.path.append('../../../../')
from sklearn.metrics import silhouette_score, davies_bouldin_score
from glob import glob
import torch
from tifffile import imread
import numpy as np
import tifffile as tiff
import os
from boilerplate import boilerplate
from lib.dataloader import CustomTestDataset
import random
from lib.evaluation import FeatureExtractor
from tqdm import tqdm

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
patch_size = 64
n_channel = 32
hierarchy_level = 3
centre_size = 4
sample_size = 1000
data_dir = "/localscratch/data/"
model = torch.load("/group/jug/Sheida/HVAE/v01/model/HVAE_best_vae.net")
model.mode_pred=True
model.eval()
model.to(device)

LadderVAE(
  (first_bottom_up): Sequential(
    (0): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ELU(alpha=1.0)
    (2): BottomUpDeterministicResBlock(
      (res): ResidualBlock(
        (block): Sequential(
          (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (1): ELU(alpha=1.0)
          (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (3): Dropout2d(p=0.2, inplace=False)
          (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (5): ELU(alpha=1.0)
          (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (7): Dropout2d(p=0.2, inplace=False)
        )
      )
    )
  )
  (top_down_layers): ModuleList(
    (0-1): 2 x TopDownLayer(
      (deterministic_block): Sequential(
        (0): TopDownDeterministicResBlock(
          (pre_conv): ConvTranspose2d(64, 64, kernel_size=(3, 3), stride=(2, 2), 

In [3]:
One_test_image = ['high_c4']

# Load test image
test_img_path = os.path.join(data_dir, One_test_image[0], f"{One_test_image[0]}_source.tif")
test_image = tiff.imread(test_img_path)

# Print loaded test images paths
print("Test image loaded from path:")
print(test_img_path)

# Load test ground truth images
test_gt_path = os.path.join(data_dir, One_test_image[0], f"{One_test_image[0]}_gt.tif")
test_ground_truth_image = tiff.imread(test_gt_path)

filtered_test_image, filtered_test_label = boilerplate._filter_slices(test_image, test_ground_truth_image)
data_mean = model.data_mean.cpu().numpy()
data_std = model.data_std.cpu().numpy()
filtered_test_image = (filtered_test_image - data_mean) / data_std
test_set = CustomTestDataset(filtered_test_image, filtered_test_label)

bg_indices = random.sample(test_set.patches_by_label[0],sample_size)
nucleus_indices = random.sample(test_set.patches_by_label[1],sample_size)
granule_indices = random.sample(test_set.patches_by_label[2],sample_size)
mito_indices = random.sample(test_set.patches_by_label[3],sample_size)

bg_samples, bg_cls, bg_lbl = test_set[bg_indices]
nucleus_samples, nucleus_cls, nucleus_lbl = test_set[nucleus_indices]
granule_samples, granule_cls, granule_lbl = test_set[granule_indices]
mito_samples, mito_cls, mito_lbl = test_set[mito_indices]

bg_samples = bg_samples.squeeze(1)
bg_lbl = bg_lbl.squeeze(1)
nucleus_samples = nucleus_samples.squeeze(1)
nucleus_lbl = nucleus_lbl.squeeze(1)
granule_samples = granule_samples.squeeze(1)
granule_lbl = granule_lbl.squeeze(1)
mito_samples = mito_samples.squeeze(1)
mito_lbl = mito_lbl.squeeze(1)


Test image loaded from path:
/localscratch/data/high_c4/high_c4_source.tif


Extracting patches from high_c4: 100%|██████████| 967/967 [00:01<00:00, 854.58it/s]


In [4]:
bg_mu, nuc_mu, gra_mu, mito_mu = np.empty((0, 43008)), np.empty((0, 43008)), np.empty((0, 43008)), np.empty((0, 43008))
FE = FeatureExtractor(model, patch_size, centre_size)
for idx in tqdm(range(sample_size)):
    mu = FE.get_feature_maps(bg_samples[idx])
    bg_mu = np.vstack([bg_mu, mu])
    mu = FE.get_feature_maps(nucleus_samples[idx])
    nuc_mu = np.vstack([nuc_mu, mu])
    mu = FE.get_feature_maps(granule_samples[idx])
    gra_mu = np.vstack([gra_mu, mu])
    mu = FE.get_feature_maps(mito_samples[idx])
    mito_mu = np.vstack([mito_mu, mu])
train_mu = np.concatenate((bg_mu, nuc_mu, gra_mu, mito_mu), axis=0)
train_labels = np.array([0] * sample_size + [1] * sample_size + [2] * sample_size + [3] * sample_size)

100%|██████████| 1000/1000 [02:42<00:00,  6.14it/s]


In [5]:
# silhouette
labels = np.array([0] * sample_size + [1] * sample_size + [2] * sample_size + [3] * sample_size)
silhouette = silhouette_score(train_mu, labels)

In [6]:
# davies bouldin
davies_bouldin = davies_bouldin_score(train_mu, labels)
print(f"Silhouette: {silhouette} Davies Bouldin: {davies_bouldin}")

Silhouette: 0.7130091161071528 Davies Bouldin: 0.5737172660313969
